In [2]:
# Cell 1: Setup and Model Loading
import os
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import gradio as gr
from google.colab import drive
import warnings
warnings.filterwarnings("ignore")

# Define model path
MODEL_PATH = "/content/drive/MyDrive/t5_summarizer_model/"

# Check if model exists
if not os.path.exists(MODEL_PATH):
    print(f"Model not found at: {MODEL_PATH}")
    print("Please check the path and ensure your model is saved there.")
else:
    print(f" Model found at: {MODEL_PATH}")

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f" Using device: {device}")

# Load tokenizer and model
print("Loading tokenizer and model...")
try:
    tokenizer = T5Tokenizer.from_pretrained(MODEL_PATH)
    model = T5ForConditionalGeneration.from_pretrained(MODEL_PATH)
    model.to(device)
    model.eval()
    print(" Model and tokenizer loaded successfully!")
    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
except Exception as e:
    print(f"Error loading model: {str(e)}")
    raise e

 Model found at: /content/drive/MyDrive/t5_summarizer_model/
 Using device: cpu
Loading tokenizer and model...
 Model and tokenizer loaded successfully!
Model parameters: 60,506,624


In [3]:
def summarize_article(article_text):
    """
    Generate summary for a given article using the fine-tuned T5 model
    """
    try:
        # Handle empty input
        if not article_text or article_text.strip() == "":
            return "⚠️ Please enter an article to summarize."

        # Clean and prepare input
        article_text = article_text.strip()

        # Format input for T5
        input_text = f"summarize: {article_text}"

        # Tokenize input
        inputs = tokenizer(
            input_text,
            return_tensors="pt",
            max_length=512,
            truncation=True,
            padding=True
        ).to(device)

        # Generate summary
        with torch.no_grad():
            outputs = model.generate(
                inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_length=64,
                min_length=10,
                num_beams=4,
                length_penalty=2.0,
                early_stopping=True,
                do_sample=False
            )

        # Decode the generated summary
        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Return formatted summary
        return f"📄 **Generated Summary:**\n\n{summary}"

    except Exception as e:
        return f"❌ Error generating summary: {str(e)}"

# Sample article for demonstration
SAMPLE_ARTICLE = """Breaking News: Scientists at MIT have developed a revolutionary new battery technology that could charge electric vehicles in under 5 minutes. The breakthrough involves using a novel lithium-metal composite that dramatically increases charging speed while maintaining safety standards. The research team, led by Dr. Sarah Chen, published their findings in the journal Nature Energy this week. Industry experts believe this technology could accelerate the adoption of electric vehicles worldwide by addressing one of the main consumer concerns about charging time. Major automotive manufacturers including Tesla, Ford, and BMW have already expressed interest in licensing the technology. The new batteries are expected to enter commercial production within the next 18 months, potentially revolutionizing the electric vehicle market and contributing significantly to global carbon reduction efforts."""

# Create Gradio interface
with gr.Blocks(
    theme=gr.themes.Soft(),
    title="News Headline Summarizer (T5 Model)",
    css="""
    .gradio-container {
        max-width: 800px !important;
        margin: 0 auto !important;
    }
    .header {
        text-align: center;
        margin-bottom: 30px;
    }
    """
) as interface:

    # Header
    gr.HTML("""
    <div class="header">
        <h1>📰 News Headline Summarizer (T5 Model)</h1>
        <p>Powered by Fine-tuned T5 Model | Generate concise summaries from news articles</p>
    </div>
    """)

    with gr.Row():
        with gr.Column():
            # Input section
            article_input = gr.TextArea(
                label="📝 Enter News Article",
                placeholder="Paste your news article here...",
                value=SAMPLE_ARTICLE,
                lines=8,
                max_lines=15
            )

            # Generate button
            generate_btn = gr.Button(
                "🚀 Generate Summary",
                variant="primary",
                scale=1
            )

            # Clear button
            clear_btn = gr.Button(
                "🗑️ Clear",
                variant="secondary",
                scale=1
            )

        with gr.Column():
            # Output section
            summary_output = gr.TextArea(
                label="📄 Generated Summary",
                placeholder="Your summary will appear here...",
                lines=6,
                max_lines=10,
                interactive=False
            )

            # Model info
            gr.HTML(f"""
            <div style="margin-top: 20px; padding: 15px; background-color: #f0f0f0; border-radius: 8px;">
                <h4>🔧 Model Information</h4>
                <p><strong>Model:</strong> Fine-tuned T5 Model</p>
                <p><strong>Device:</strong> {device}</p>
                <p><strong>Max Input Length:</strong> 512 tokens</p>
                <p><strong>Max Output Length:</strong> 64 tokens</p>
            </div>
            """)

    # Event handlers
    generate_btn.click(
        fn=summarize_article,
        inputs=article_input,
        outputs=summary_output
    )

    clear_btn.click(
        fn=lambda: ("", ""),
        inputs=None,
        outputs=[article_input, summary_output]
    )

    # Examples
    gr.Examples(
        examples=[
            [SAMPLE_ARTICLE],
            ["Breaking: Apple announces new iPhone with revolutionary camera technology that can capture photos in complete darkness using advanced AI algorithms."],
            ["Climate scientists warn that global temperatures are rising faster than predicted, with Arctic ice melting at an unprecedented rate according to new NASA satellite data."]
        ],
        inputs=article_input,
        outputs=summary_output,
        fn=summarize_article,
        cache_examples=False
    )

# Launch the interface
print("🚀 Launching Gradio Interface...")
interface.launch(
    share=True,
    debug=False,
    show_error=True,
    server_port=7860,
    height=600
)

print("✅ Interface launched successfully!")
print("🔗 You can now use the interface to summarize news articles!")

🚀 Launching Gradio Interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ceacc2e417890d8fd3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


✅ Interface launched successfully!
🔗 You can now use the interface to summarize news articles!
